In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
load_dotenv(override=True)
engine = create_engine(os.environ.get("ALCHEMY_DATABASE_URL"), connect_args={'connect_timeout': 600})

## Query Function

In [2]:
#function definition for running queries
def run_query(query):
    with engine.connect() as connection:
        return pd.read_sql(query, connection)

## Roll Up

In [3]:
# Roll Up
roll_up_query = """
SELECT D.year, COUNT(G.id) AS total_games 
FROM game_fact_table AS G 
JOIN dimDate D ON G.releaseDate = D.date 
GROUP BY D.year;
"""
roll_up_df = run_query(roll_up_query)
print("Roll Up Query Result")
print(roll_up_df)

Roll Up Query Result
    year  total_games
0   1997            2
1   1998            1
2   1999            3
3   2000            2
4   2001            4
5   2002            1
6   2003            3
7   2004            7
8   2005            7
9   2006           69
10  2007          109
11  2008          176
12  2009          343
13  2010          309
14  2011          286
15  2012          357
16  2013          492
17  2014         1634
18  2015         2693
19  2016         4416
20  2017         6334
21  2018         8183
22  2019         7827
23  2020         9647
24  2021        12392
25  2022        13989
26  2023        15542
27  2024        12580
28  2025            2


## Dice

In [4]:
# Dice
dice_query = """
SELECT name, price, metacriticScore 
FROM game_fact_table 
WHERE price > 20 AND metacriticScore > 80;
"""
dice_df = run_query(dice_query)
print("Dice Query Result")
print(dice_df)

Dice Query Result
                                       name  price  metacriticScore
0          Medieval II: Total War™ Kingdoms  24.99               85
1                         BioShock Infinite  29.99               94
2               Sid Meier's Civilization® V  29.99               90
3              Borderlands Game of the Year  29.99               81
4    Total War: EMPIRE – Definitive Edition  24.99               90
..                                      ...    ...              ...
235                  Dungeons of Hinterberg  26.99               82
236                           Blasphemous 2  29.99               83
237                        Magical Delicacy  24.99               84
238                                  Ultros  22.49               83
239       Kunitsu-Gami: Path of the Goddess  49.99               84

[240 rows x 3 columns]


## Drill Down

In [6]:
# Drill Down

drill_down_query = """
SELECT G.name, G.reviews, d.name AS developer 
FROM Game_Fact_Table AS G, developer AS d
JOIN dimDeveloper ON d.groupID = dimDeveloper.dimDeveloperId
WHERE D.name = 'Valve';
"""
drill_down_df = run_query(drill_down_query)
print("Drill Down Query Result")
print(drill_down_df)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'D.name' in 'field list'
[SQL: 
SELECT G.name, G.reviews, D.name AS developer 
FROM Game_Fact_Table AS G
JOIN dimDeveloper D ON G.dimDeveloperId = D.dimDeveloperId
WHERE D.name = 'Valve';
]
(Background on this error at: https://sqlalche.me/e/20/f405)

## Slice

In [7]:
# Slice
slice_query = """
SELECT name, releaseDate, price 
FROM game_fact_table 
WHERE releaseDate >= '2022-01-01';
"""
slice_df = run_query(slice_query)
print("Slice Query Result")
print(slice_df)

Slice Query Result
                               name releaseDate  price
0      Borderlands Game of the Year  2023-08-31  29.99
1           Train Simulator Classic  2022-04-21  24.99
2           Train Simulator Classic  2022-04-21  24.99
3           Train Simulator Classic  2022-04-21  24.99
4           Train Simulator Classic  2022-04-21  24.99
...                             ...         ...    ...
42108        Storm Cleaner Playtest  2024-08-27   0.00
42109  Spaceborne Fighters Playtest  2024-08-31   0.00
42110            NET.CRAWL Playtest  2024-08-30   0.00
42111    Food Truck Empire Playtest  2024-08-30   0.00
42112               Carbon Playtest  2024-09-01   0.00

[42113 rows x 3 columns]
